In [ ]:
import numpy as np
from numpy.lib.stride_tricks import as_strided
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

from PIL import Image
from skimage.color import rgb2gray
from scipy.ndimage import zoom

import tensorflow as tf
from tensorflow import keras

In [ ]:
seismic_rgb = Image.open('seismic.jpg')

In [ ]:
image_original = seismic_rgb.resize((4000, 2000), Image.ANTIALIAS)

In [ ]:
seismic_gray_original = rgb2gray(image_original)

In [ ]:
seismic_sliced_original = seismic_gray_original[600:1370, 500:3500]

In [ ]:
def standardization(data):
    positive = data + (-1 * np.amin(data))
    norm = positive / np.amax(positive)
    center = norm - np.mean(norm)
    standard = (center - np.mean(center)) / np.std(center, dtype=np.float32)
    return standard

In [ ]:
seismic_std_original = standardization(seismic_sliced_original)

In [ ]:
path = 'to_model'
model = tf.keras.models.load_model(f'{path}/SRT-Ai/')

In [ ]:
def sliding_window_view(arr, window_shape, step):
    """
    Generate a view of 'arr' with windows of 'window_shape' moved by 'step'.
    Parameters:
    arr: Input array.
    window_shape: The shape of the desired window.
    step: The step size or stride between windows in each dimension.
 
    Returns:
    A view into 'arr' with the given window shape and step.
    """
 
    arr_shape = np.array(arr.shape)
    window_shape = np.array(window_shape)
    steps = np.array(step)
 
    # Calculate new_shape considering the step and window shape
    new_shape = ((arr_shape - window_shape) // steps) + 1
 
    # Calculate new_strides considering the original strides and the step
    new_strides = tuple(arr.strides * steps)
 
    # Adjust shape and strides for as_strided
    adjusted_shape = tuple(new_shape) + tuple(window_shape)
    adjusted_strides = tuple(new_strides) + tuple(arr.strides)
 
    return as_strided(arr, shape=adjusted_shape, strides=adjusted_strides)

In [ ]:
# Test the function with the specified parameters
array_original = seismic_std_original.T
window_shape = (200, 99)                            # Shape of the window
step = (10, 3)                                      # Step size
 
# Generate the windows
windows_original = sliding_window_view(array_original, window_shape, step)
 
# Output the shape of the resulting windows array
windows_original.shape

In [ ]:
# Original size predictions:

predictions_original = []

for i in range(len(windows_original[0])):
    for j in range(len(windows_original[1])):
        array_original = windows_original[i,j].T
        expanded_original = np.expand_dims(array_original, axis=0)
        expanded_original = np.expand_dims(expanded_original, axis=3)
        pred = model.predict(expanded_original)
        predictions_original.append(pred);

In [ ]:
pred_array = np.array(predictions_original)

In [ ]:
pred_CTclass = pred_array.reshape(len(windows_original[0]), len(windows_original[1]), 2)[:,:,1]

In [ ]:
def resample_array(source_array, target_array):
    # Calculate the scaling factors for each dimension
    scaling_factors = np.array(target_array.shape) / np.array(source_array.shape)
    
    # Resample the source array to match the target array shape
    resampled_array = zoom(source_array, zoom=scaling_factors, order=3, mode='reflect')
    
    return resampled_array

In [ ]:
resampled_pred_CTclass = resample_array(pred_CTclass.T, seismic_std_original)

In [ ]:
# Transparency:
alphas = Normalize(0, 0.1, clip=True)(np.abs(resampled_pred_CTclass))

plt.figure(figsize=(20, 10))
plt.imshow(seismic_std_original, 'gray', vmin = -1, vmax = 1)
plt.imshow(resampled_resampled_pred_CTclass, vmin = 0, vmax = 1, alpha=alphas*0.4)
plt.colorbar(shrink=0.4)

plt.show()